In [20]:
from pyspark.sql import SparkSession

try:
  spark = SparkSession.builder.getOrCreate()
  sc = spark.sparkContext
except:
  print("Already running.")

In [27]:
import json

exampleJsonRecord = {
  "id": "1",
  "name": "Connor",
  "gender": "CIS-MALE",
  "isGraduated": True,
  "createdAt": "2023-01-18T00:00:01Z",
  "languages": [
    "English"
  ],
  "programmingLanguages": [
    "Java",
    "C++",
    "TypeScript",
    "JavaScript",
    "Python",
    "SQL"
  ],
  "workExperiences": [
    {
      "company": "John Deere",
      "location": "Urbandale",
      "years": 5
    }
  ]
}
jsonStrings = [json.dumps(exampleJsonRecord)]
jsonStrings

['{"id": "1", "name": "Connor", "gender": "CIS-MALE", "isGraduated": true, "createdAt": "2023-01-18T00:00:01Z", "languages": ["English"], "programmingLanguages": ["Java", "C++", "TypeScript", "JavaScript", "Python", "SQL"], "workExperiences": [{"company": "John Deere", "location": "Urbandale", "years": 5}]}']

In [28]:
peopleRDD = sc.parallelize(jsonStrings)
people = spark.read.json(peopleRDD)
people.show()

+--------------------+--------+---+-----------+---------+------+--------------------+--------------------+
|           createdAt|  gender| id|isGraduated|languages|  name|programmingLanguages|     workExperiences|
+--------------------+--------+---+-----------+---------+------+--------------------+--------------------+
|2023-01-18T00:00:01Z|CIS-MALE|  1|       true|[English]|Connor|[Java, C++, TypeS...|[{John Deere, Urb...|
+--------------------+--------+---+-----------+---------+------+--------------------+--------------------+



In [29]:
people.createOrReplaceGlobalTempView("people")

In [30]:
spark.sql("""
  SELECT * FROM global_temp.people
""").show()

+--------------------+--------+---+-----------+---------+------+--------------------+--------------------+
|           createdAt|  gender| id|isGraduated|languages|  name|programmingLanguages|     workExperiences|
+--------------------+--------+---+-----------+---------+------+--------------------+--------------------+
|2023-01-18T00:00:01Z|CIS-MALE|  1|       true|[English]|Connor|[Java, C++, TypeS...|[{John Deere, Urb...|
+--------------------+--------+---+-----------+---------+------+--------------------+--------------------+



In [54]:

from pyspark.sql.functions import explode,map_keys,col,flatten

companies = people.select(explode(people.workExperiences), col("col").alias("companies")).drop("col")
companies.select(col("companies.*")).createOrReplaceGlobalTempView("companies")

languages = people.select(explode(people.languages), col("col").alias("languages")).drop("col")
languages.createOrReplaceGlobalTempView("languages")

programmingLanguages = people.select(explode(people.programmingLanguages), col("col").alias("programmingLanguages")).drop("col")
programmingLanguages.createOrReplaceGlobalTempView("programmingLanguages")

In [51]:
spark.sql("""
  SELECT * FROM global_temp.companies
""").show()

+----------+---------+-----+
|   company| location|years|
+----------+---------+-----+
|John Deere|Urbandale|    5|
+----------+---------+-----+



In [55]:
spark.sql("""
  SELECT * FROM global_temp.languages
""").show()

+---------+
|languages|
+---------+
|  English|
+---------+



In [56]:
spark.sql("""
  SELECT * FROM global_temp.programmingLanguages
""").show()

+--------------------+
|programmingLanguages|
+--------------------+
|                Java|
|                 C++|
|          TypeScript|
|          JavaScript|
|              Python|
|                 SQL|
+--------------------+

